dataset_two 학습

https://www.kaggle.com/datasets/glebkuzntesov/top-view-drone-car-detection-dataset-12000-images

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 42.0 MB/s eta 0:00:00


In [ ]:
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
model = YOLO("/content/drive/MyDrive/best_s.pt")

In [ ]:
model = YOLO("/content/drive/MyDrive/졸작/train_results/dataset_two_train/weights/last.pt")

In [ ]:
# 이어서 학습
model.train(
    data="/content/drive/MyDrive/졸작/dataset_two/dataset.yaml",

    epochs=20,   # 총 epoch 목표 (11부터 이어서 20까지 감)
    imgsz=640,
    batch=16,
    device=0,
    project="/content/drive/MyDrive/졸작/train_results",
    name="dataset_two_train",
    resume=True   # 이어서 학습 옵션
)


Ultralytics 8.3.199 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/졸작/dataset_two/dataset.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/content/drive/MyDrive/졸작/train_results/dataset_two_train/weights/last.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=dataset_two_train, nbs=64, nms=False, opset=None, optimiz

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7e4357e3d9a0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

증강 데이터로 파인튜닝

In [ ]:
import os
import random
import shutil
import yaml
from google.colab import drive

# 1. 구글 드라이브 연결
drive.mount('/content/drive')

# 2. 데이터 경로 설정 (수정하세요!)
image_dir = "/content/drive/MyDrive/졸작/parking_data/augmented_images/top_view/train/images"
label_dir = "/content/drive/MyDrive/졸작/parking_data/augmented_images/top_view/train/annotations"
output_dir = "/content/drive/MyDrive/졸작/parking_data/dataset_split"  # 결과 저장 폴더

# 3. train/val 비율
val_ratio = 0.2

# 4. train/val 폴더 안에 images/labels 폴더 만들기
for split in ["train", "val"]:
    os.makedirs(os.path.join(output_dir, split, "images"), exist_ok=True)
    os.makedirs(os.path.join(output_dir, split, "labels"), exist_ok=True)

# 5. 이미지/라벨 파일 리스트 가져오기
images = [f for f in os.listdir(image_dir) if f.lower().endswith(('.jpg', '.png', '.jpeg'))]
random.shuffle(images)

# train/val 나누기
split_idx = int(len(images) * (1 - val_ratio))
train_files = images[:split_idx]
val_files = images[split_idx:]

# 6. 파일 복사 함수
def copy_files(file_list, split):
    for img_file in file_list:
        # 이미지
        src_img = os.path.join(image_dir, img_file)
        dst_img = os.path.join(output_dir, split, "images", img_file)
        shutil.copy(src_img, dst_img)

        # 라벨 (같은 이름, 확장자만 .txt)
        label_file = os.path.splitext(img_file)[0] + ".txt"
        src_label = os.path.join(label_dir, label_file)
        dst_label = os.path.join(output_dir, split, "labels", label_file)

        if os.path.exists(src_label):
            shutil.copy(src_label, dst_label)

# 7. train/val 복사
copy_files(train_files, "train")
copy_files(val_files, "val")

print(f"✅ Train: {len(train_files)}개, Val: {len(val_files)}개 분할 완료!")

# 8. dataset.yaml 생성
yaml_path = os.path.join(output_dir, "dataset.yaml")
dataset_yaml = {
    "train": os.path.join(output_dir, "train", "images"),
    "val": os.path.join(output_dir, "val", "images"),
    "nc": 1,  # 클래스 개수
    "names": ["vehicle"]  # 클래스 이름
}

with open(yaml_path, "w") as f:
    yaml.dump(dataset_yaml, f)

print(f"✅ dataset.yaml 생성 완료 → {yaml_path}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Train: 1385개, Val: 347개 분할 완료!
✅ dataset.yaml 생성 완료 → /content/drive/MyDrive/졸작/parking_data/dataset_split/dataset.yaml


In [ ]:
import os

# YOLO 라벨 폴더 경로
label_dirs = [
    "/content/drive/MyDrive/졸작/parking_data/dataset_split/train/labels",
    "/content/drive/MyDrive/졸작/parking_data/dataset_split/val/labels"
]

for labels_dir in label_dirs:
    for filename in os.listdir(labels_dir):
        if filename.endswith(".txt"):
            filepath = os.path.join(labels_dir, filename)
            with open(filepath, "r") as f:
                lines = f.readlines()

            # class 1만 제거, class 0은 유지
            new_lines = [line for line in lines if not line.startswith("1 ")]

            with open(filepath, "w") as f:
                f.writelines(new_lines)

print("모든 train/test 라벨에서 Class 1 제거 완료! Class 0은 그대로 남았습니다.")


모든 train/test 라벨에서 Class 1 제거 완료! Class 0은 그대로 남았습니다.


In [ ]:
# 3️⃣ 모델 불러오기 (기존 best.pt)
model_path = "/content/drive/MyDrive/졸작/train_results/dataset_two_train/weights/best.pt"
model = YOLO(model_path)

# 4️⃣ 학습용 데이터셋 경로 (dataset.yaml)
data_yaml = "/content/drive/MyDrive/졸작/parking_data/dataset_split/dataset.yaml"

# 5️⃣ 파인튜닝 학습
model.train(
    data=data_yaml,
    epochs=20,
    batch=16,
    imgsz=640,
    device=0,
    project="/content/drive/MyDrive/졸작/train_results_dataset_split",
    name="fine_tune_augmented",
    pretrained=True
)

Ultralytics 8.3.200 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/졸작/parking_data/dataset_split/dataset.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/content/drive/MyDrive/졸작/train_results/dataset_two_train/weights/best.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=fine_tune_augmented, nbs=64, nms=False, o

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7d233f0139e0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 